In [2]:
# Valuation of European Call Options using Lewis (2001) Fourier method
# Models: BSM and H93

#Import necessary packages
import numpy as np
from scipy.integrate import quad
from scipy import stats
import warnings
warnings.simplefilter('ignore')

#Lewis (2001) fourier based integration approach

def BSM_call_value(S0, K, T, r, d, sigma):
    ''' Valuation of European call option in BSM model via Lewis (2001)
    --> Fourier-based approach (integral).
    Parameters
    ==========
    S0: float
        initial stock/index level
    K: float
        strike price
    T: float
        time-to-maturity (for t=0)
    r: float
        constant risk-free short rate
    d: float
        dividend yield
    sigma: float
        volatility factor in diffusion term
    Returns
    =======
    call_value: float
        European call option present value
    '''
    int_value = quad(lambda u:
               BSM_integral_function(u, S0, K, T, r, d, sigma), 0, 100)[0]
    call_value = max(0, S0*np.exp(-d*T) - np.exp(-(r+d) * T/2) * np.sqrt(S0 * K)
                                                / np.pi * int_value)
    return call_value


def H93_call_value(S0, K, T, r, d, kappa_v, theta_v, sigma_v, rho, v0):
    ''' Valuation of European call option in H93 model via Lewis (2001)
    Fourier-based approach.
    Parameters
    ==========
    S0: float
        initial stock/index level
    K: float
        strike price
    T: float
        time-to-maturity (for t=0)
    r: float
        constant risk-free short rate
     d: float
        dividend yield
    kappa_v: float
        mean-reversion factor
    theta_v: float
        long-run mean of variance
    sigma_v: float
        volatility of variance
    rho: float
        correlation between variance and stock/index level
    v0: float
        initial level of variance
    Returns
    =======
    call_value: float
        present value of European call option
    '''
    int_value = quad(lambda u: H93_int_func(u, S0, K, T, r, d, kappa_v,
                        theta_v, sigma_v, rho, v0), 0, np.inf, limit=250)[0]
    call_value = max(0, S0*np.exp(-d*T) - np.exp(-(r+d) * T/2) * np.sqrt(S0 * K)
                                                / np.pi * int_value)
    return call_value


#Integration Functions

def BSM_integral_function(u, S0, K, T, r, d, sigma):
    ''' Valuation of European call option in BSM model via Lewis (2001)
    --> Fourier-based approach: integral function. '''
    cf_value = BSM_characteristic_function(u - 1j * 0.5, T, r, d, sigma)
    int_value = 1 / (u ** 2 + 0.25) \
            * (np.exp(1j * u * (np.log(S0 / K)+(r-d)*T)) * cf_value).real
    return int_value


def H93_int_func(u, S0, K, T, r, d, kappa_v, theta_v, sigma_v, rho, v0):
    ''' Valuation of European call option in H93 model via Lewis (2001)
    Fourier-based approach: integration function.
    Parameter definitions see function H93_call_value.'''
    char_func_value = H93_char_func(u - 1j * 0.5, T, r, d, kappa_v,
                                    theta_v, sigma_v, rho, v0)
    int_func_value = 1 / (u ** 2 + 0.25) \
            * (np.exp(1j * u * (np.log(S0 / K)+(r-d)*T)) * char_func_value).real
    return int_func_value


#Characteristic Functions

def BSM_characteristic_function(v, T, r, d, sigma):
    '''  Valuation of European call option in BSM model via
    Lewis (2001) and Carr-Madan (1999)
    --> Fourier-based approach: charcteristic function. '''
    cf_value = np.exp(((- 0.5 * sigma ** 2) * 1j * v
                - 0.5 * sigma ** 2 * v ** 2) * T)
    return cf_value


def H93_char_func(u, T, r, d, kappa_v, theta_v, sigma_v, rho, v0):
    ''' Valuation of European call option in H93 model via Lewis (2001)
    Fourier-based approach: characteristic function.
    Parameter definitions see function BCC_call_value.'''
    c1 = kappa_v * theta_v
    c2 = -np.sqrt((rho * sigma_v * u * 1j - kappa_v)
            ** 2 - sigma_v ** 2 * (-u * 1j - u ** 2))
    c3 = (kappa_v - rho * sigma_v * u * 1j + c2) \
          / (kappa_v - rho * sigma_v * u * 1j - c2)
    H1 = ((r-d) * u * 1j * T + (c1 / sigma_v ** 2)
          * ((kappa_v - rho * sigma_v * u * 1j + c2) * T
                - 2 * np.log((1 - c3 * np.exp(c2 * T)) / (1 - c3))))
    H2 = ((kappa_v - rho * sigma_v * u * 1j + c2) / sigma_v ** 2
          * ((1 - np.exp(c2 * T)) / (1 - c3 * np.exp(c2 * T))))
    char_func_value = np.exp(H1 + H2 * v0)*np.exp(- 1j*u*(r-d)*T)
    return char_func_value

